In [1]:
import numpy as np
import sympy as sm
import sympy.physics.mechanics as me
me.init_vprinting(use_latex='mathjax')

In [2]:
class ReferenceFrame(me.ReferenceFrame):

    def __init__(self, *args, **kwargs):

        kwargs.pop('latexs', None)

        lab = args[0].lower()
        tex = r'\hat{{{}}}_{}'

        super(ReferenceFrame, self).__init__(*args,
                                             latexs=(tex.format(lab, 'x'),
                                                     tex.format(lab, 'y'),
                                                     tex.format(lab, 'z')),
                                             **kwargs)
me.ReferenceFrame = ReferenceFrame

In [3]:
m1, m2, l1, l2, g = sm.symbols('m1, m2, l1, l2, g')
q1, q2, u1, u2, T1, T2 = me.dynamicsymbols('q1, q2, u1, u2, T1, T2')
t = me.dynamicsymbols._t

p = sm.Matrix([m1, m2, l1, l2, g])
q = sm.Matrix([q1, q2])
u = sm.Matrix([u1, u2])
r = sm.Matrix([T1, T2])

ud = u.diff(t)

p, q, u, r, ud

⎛⎡m₁⎤                        ⎞
⎜⎢  ⎥                        ⎟
⎜⎢m₂⎥                        ⎟
⎜⎢  ⎥  ⎡q₁⎤  ⎡u₁⎤  ⎡T₁⎤  ⎡u₁̇⎤⎟
⎜⎢l₁⎥, ⎢  ⎥, ⎢  ⎥, ⎢  ⎥, ⎢  ⎥⎟
⎜⎢  ⎥  ⎣q₂⎦  ⎣u₂⎦  ⎣T₂⎦  ⎣u₂̇⎦⎟
⎜⎢l₂⎥                        ⎟
⎜⎢  ⎥                        ⎟
⎝⎣g ⎦                        ⎠

In [4]:
N = me.ReferenceFrame('N')
A = me.ReferenceFrame('A')
B = me.ReferenceFrame('B')

A.orient_axis(N, q1, N.z)
B.orient_axis(N, q2, N.z)

A.set_ang_vel(N, u1*N.z)
B.set_ang_vel(N, u2*N.z)

In [5]:
O = me.Point('O')
P1 = O.locatenew('P1', -l1*A.y)
P2 = P1.locatenew('P2', -l2*B.y)

O.set_vel(N, 0)
P1.v2pt_theory(O, N, A)

l₁⋅u₁ a_x

In [6]:
P2.v2pt_theory(P1, N, B)

l₁⋅u₁ a_x + l₂⋅u₂ b_x

In [7]:
P1.a2pt_theory(O, N, A)

                 2
l₁⋅u₁̇ a_x + l₁⋅u₁  a_y

In [8]:
P2.a2pt_theory(P1, N, B)

                 2                        2
l₁⋅u₁̇ a_x + l₁⋅u₁  a_y + l₂⋅u₂̇ b_x + l₂⋅u₂  b_y

In [9]:
F_P1 = T1*A.y - T2*B.y - m1*g*N.y
F_P1.express(N)

(-T₁⋅sin(q₁) + T₂⋅sin(q₂)) n_x + (-g⋅m₁ + T₁⋅cos(q₁) - T₂⋅cos(q₂)) n_y

In [10]:
F_P2 = T2*B.y - m2*g*N.y
F_P2.express(N)

-T₂⋅sin(q₂) n_x + (-g⋅m₂ + T₂⋅cos(q₂)) n_y

In [11]:
zero_P1 = F_P1 - m1*P1.acc(N)
zero_P2 = F_P2 - m2*P2.acc(N)

In [12]:
fd = sm.Matrix([
    zero_P1.dot(N.x),
    zero_P1.dot(N.y),
    zero_P2.dot(N.x),
    zero_P2.dot(N.y),
])
fd

⎡                                    ⎛          2     ⎞                       
⎢                -l₁⋅m₁⋅cos(q₁)⋅u₁̇ - ⎝- l₁⋅m₁⋅u₁  + T₁⎠⋅sin(q₁) + T₂⋅sin(q₂) 
⎢                                                                             
⎢                                        ⎛          2     ⎞                   
⎢             -g⋅m₁ - l₁⋅m₁⋅sin(q₁)⋅u₁̇ + ⎝- l₁⋅m₁⋅u₁  + T₁⎠⋅cos(q₁) - T₂⋅cos(
⎢                                                                             
⎢            2                                                 ⎛          2   
⎢    l₁⋅m₂⋅u₁ ⋅sin(q₁) - l₁⋅m₂⋅cos(q₁)⋅u₁̇ - l₂⋅m₂⋅cos(q₂)⋅u₂̇ - ⎝- l₂⋅m₂⋅u₂  
⎢                                                                             
⎢                2                                                 ⎛          
⎣-g⋅m₂ - l₁⋅m₂⋅u₁ ⋅cos(q₁) - l₁⋅m₂⋅sin(q₁)⋅u₁̇ - l₂⋅m₂⋅sin(q₂)⋅u₂̇ + ⎝- l₂⋅m₂⋅

               ⎤
                ⎥
               ⎥
               ⎥
q₂)             ⎥
               ⎥
  ⎞            ⎥
+ T₂⎠⋅sin

In [13]:
(me.find_dynamicsymbols(fd[0]), me.find_dynamicsymbols(fd[1]),
 me.find_dynamicsymbols(fd[2]), me.find_dynamicsymbols(fd[3]))

({T₁, T₂, q₁, q₂, u₁, u₁̇}, {T₁, T₂, q₁, q₂, u₁, u₁̇}, {T₂, q₁, q₂, u₁, u₂, u₁
̇, u₂̇}, {T₂, q₁, q₂, u₁, u₂, u₁̇, u₂̇})

In [14]:
ud, r

⎛⎡u₁̇⎤  ⎡T₁⎤⎞
⎜⎢  ⎥, ⎢  ⎥⎟
⎝⎣u₂̇⎦  ⎣T₂⎦⎠

In [15]:
udr = ud.col_join(r)
udr_zero = {v: 0 for v in udr}

Md = fd.jacobian(udr)
gd = fd.xreplace(udr_zero)

Md, udr, gd

⎛                                                            ⎡                
⎜                                                            ⎢              l₁
⎜⎡-l₁⋅m₁⋅cos(q₁)        0         -sin(q₁)  sin(q₂) ⎤  ⎡u₁̇⎤  ⎢               
⎜⎢                                                  ⎥  ⎢  ⎥  ⎢                
⎜⎢-l₁⋅m₁⋅sin(q₁)        0         cos(q₁)   -cos(q₂)⎥  ⎢u₂̇⎥  ⎢          -g⋅m₁
⎜⎢                                                  ⎥, ⎢  ⎥, ⎢                
⎜⎢-l₁⋅m₂⋅cos(q₁)  -l₂⋅m₂⋅cos(q₂)     0      -sin(q₂)⎥  ⎢T₁⎥  ⎢            2   
⎜⎢                                                  ⎥  ⎢  ⎥  ⎢    l₁⋅m₂⋅u₁ ⋅si
⎜⎣-l₁⋅m₂⋅sin(q₁)  -l₂⋅m₂⋅sin(q₂)     0      cos(q₂) ⎦  ⎣T₂⎦  ⎢                
⎜                                                            ⎢                
⎝                                                            ⎣-g⋅m₂ - l₁⋅m₂⋅u₁

      2                      ⎤⎞
⋅m₁⋅u₁ ⋅sin(q₁)              ⎥⎟
                              ⎥⎟
          2                  ⎥⎟
 

In [16]:
u3, u4 = me.dynamicsymbols('u3, u4')

N_v_P1a = P1.vel(N) - u3*A.y
N_v_P1a

l₁⋅u₁ a_x + -u₃ a_y

In [17]:
N_v_P2a = N_v_P1a + me.cross(B.ang_vel_in(N), P2.pos_from(P1)) - u4*B.y
N_v_P2a

l₁⋅u₁ a_x + -u₃ a_y + l₂⋅u₂ b_x + -u₄ b_y

In [18]:
R_P1 = -m1*g*N.y
R_P2 = -m2*g*N.y

In [19]:
F1 = P1.vel(N).diff(u1, N).dot(R_P1) + P2.vel(N).diff(u1, N).dot(R_P2)
F1

-g⋅l₁⋅m₁⋅sin(q₁) - g⋅l₁⋅m₂⋅sin(q₁)

In [20]:
F2 = P1.vel(N).diff(u2, N).dot(R_P1) + P2.vel(N).diff(u2, N).dot(R_P2)
F2

-g⋅l₂⋅m₂⋅sin(q₂)

In [21]:
R_P1_aux = R_P1 + T1*A.y - T2*B.y
R_P2_aux = R_P2 + T2*B.y

In [22]:
F3 = N_v_P1a.diff(u3, N).dot(R_P1_aux) + N_v_P2a.diff(u3, N).dot(R_P2_aux)
F3

g⋅m₁⋅cos(q₁) + g⋅m₂⋅cos(q₁) - T₁

In [23]:
F4 = N_v_P1a.diff(u4, N).dot(R_P1_aux) + N_v_P2a.diff(u4, N).dot(R_P2_aux)
F4

g⋅m₂⋅cos(q₂) - T₂

In [24]:
Fr = sm.Matrix([F1, F2, F3, F4])
Fr

⎡-g⋅l₁⋅m₁⋅sin(q₁) - g⋅l₁⋅m₂⋅sin(q₁)⎤
⎢                                  ⎥
⎢         -g⋅l₂⋅m₂⋅sin(q₂)         ⎥
⎢                                  ⎥
⎢ g⋅m₁⋅cos(q₁) + g⋅m₂⋅cos(q₁) - T₁ ⎥
⎢                                  ⎥
⎣        g⋅m₂⋅cos(q₂) - T₂         ⎦

In [25]:
Rs_P1 = -m1*P1.acc(N)
Rs_P2 = -m2*P2.acc(N)

In [26]:
F1s = P1.vel(N).diff(u1, N).dot(Rs_P1) + P2.vel(N).diff(u1, N).dot(Rs_P2)
F1s

    2           2            ⎛                                                
- l₁ ⋅m₁⋅u₁̇ - l₁ ⋅m₂⋅u₁̇ + l₁⋅⎝-l₂⋅m₂⋅(sin(q₁)⋅sin(q₂) + cos(q₁)⋅cos(q₂))⋅u₂̇

                                            2⎞
 - l₂⋅m₂⋅(sin(q₁)⋅cos(q₂) - sin(q₂)⋅cos(q₁))⋅u₂ ⎠

In [27]:
F2s = P1.vel(N).diff(u2, N).dot(Rs_P1) + P2.vel(N).diff(u2, N).dot(Rs_P2)
F2s

    2            ⎛                                                            
- l₂ ⋅m₂⋅u₂̇ + l₂⋅⎝-l₁⋅m₂⋅(sin(q₁)⋅sin(q₂) + cos(q₁)⋅cos(q₂))⋅u₁̇ - l₁⋅m₂⋅(-si

                                 2⎞
n(q₁)⋅cos(q₂) + sin(q₂)⋅cos(q₁))⋅u₁ ⎠

In [28]:
F3s = N_v_P1a.diff(u3, N).dot(Rs_P1) + N_v_P2a.diff(u3, N).dot(Rs_P2)
F3s

        2           2                                               2         
l₁⋅m₁⋅u₁  + l₁⋅m₂⋅u₁  + l₂⋅m₂⋅(sin(q₁)⋅sin(q₂) + cos(q₁)⋅cos(q₂))⋅u₂  + l₂⋅m₂⋅

                                       
(-sin(q₁)⋅cos(q₂) + sin(q₂)⋅cos(q₁))⋅u₂̇

In [29]:
F4s = N_v_P1a.diff(u4, N).dot(Rs_P1) + N_v_P2a.diff(u4, N).dot(Rs_P2)
F4s

                                            2                                 
l₁⋅m₂⋅(sin(q₁)⋅sin(q₂) + cos(q₁)⋅cos(q₂))⋅u₁  + l₁⋅m₂⋅(sin(q₁)⋅cos(q₂) - sin(q

                         2
₂)⋅cos(q₁))⋅u₁̇ + l₂⋅m₂⋅u₂ 

In [30]:
Frs = sm.Matrix([F1s, F2s, F3s, F4s])
Frs = sm.trigsimp(Frs)
Frs

⎡    ⎛                              2                                     ⎞⎤
⎢-l₁⋅⎝l₁⋅m₁⋅u₁̇ + l₁⋅m₂⋅u₁̇ + l₂⋅m₂⋅u₂ ⋅sin(q₁ - q₂) + l₂⋅m₂⋅cos(q₁ - q₂)⋅u₂̇⎠⎥
⎢                                                                          ⎥
⎢               ⎛     2                                          ⎞         ⎥
⎢         l₂⋅m₂⋅⎝l₁⋅u₁ ⋅sin(q₁ - q₂) - l₁⋅cos(q₁ - q₂)⋅u₁̇ - l₂⋅u₂̇⎠         ⎥
⎢                                                                          ⎥
⎢          2           2           2                                       ⎥
⎢  l₁⋅m₁⋅u₁  + l₁⋅m₂⋅u₁  + l₂⋅m₂⋅u₂ ⋅cos(q₁ - q₂) - l₂⋅m₂⋅sin(q₁ - q₂)⋅u₂̇  ⎥
⎢                                                                          ⎥
⎢             ⎛     2                                          2⎞          ⎥
⎣          m₂⋅⎝l₁⋅u₁ ⋅cos(q₁ - q₂) + l₁⋅sin(q₁ - q₂)⋅u₁̇ + l₂⋅u₂ ⎠          ⎦

In [31]:
fa = Frs + Fr
me.find_dynamicsymbols(fa)

{T₁, T₂, q₁, q₂, u₁, u₂, u₁̇, u₂̇}

In [32]:
Ma = fa.jacobian(udr)
ga = fa.xreplace(udr_zero)

Ma, udr, ga

⎛                                                                ⎡            
⎜⎡ -l₁⋅(l₁⋅m₁ + l₁⋅m₂)    -l₁⋅l₂⋅m₂⋅cos(q₁ - q₂)  0   0 ⎤        ⎢       -g⋅l₁
⎜⎢                                                      ⎥  ⎡u₁̇⎤  ⎢           
⎜⎢                                  2                   ⎥  ⎢  ⎥  ⎢            
⎜⎢-l₁⋅l₂⋅m₂⋅cos(q₁ - q₂)         -l₂ ⋅m₂          0   0 ⎥  ⎢u₂̇⎥  ⎢           
⎜⎢                                                      ⎥, ⎢  ⎥, ⎢            
⎜⎢          0              -l₂⋅m₂⋅sin(q₁ - q₂)    -1  0 ⎥  ⎢T₁⎥  ⎢            
⎜⎢                                                      ⎥  ⎢  ⎥  ⎢g⋅m₁⋅cos(q₁)
⎜⎣  l₁⋅m₂⋅sin(q₁ - q₂)              0             0   -1⎦  ⎣T₂⎦  ⎢            
⎜                                                                ⎢            
⎝                                                                ⎣            

                                           2                    ⎤⎞
⋅m₁⋅sin(q₁) - g⋅l₁⋅m₂⋅sin(q₁) - l₁⋅l₂⋅m₂⋅u₂ ⋅sin(q₁ - q₂)      

In [33]:
udr_sol = -Ma.LUsolve(ga)

In [34]:
T1_sol = sm.trigsimp(udr_sol[2])
T1_sol

                                                                              
                                                                              
                                                                              
                                      2           2           2               
g⋅m₁⋅cos(q₁) + g⋅m₂⋅cos(q₁) + l₁⋅m₁⋅u₁  + l₁⋅m₂⋅u₁  + l₂⋅m₂⋅u₂ ⋅cos(q₁ - q₂) +
                                                                              
                                                                              
                                                                              
                                                                              

       ⎛                                                        ⎛             
       ⎜                              2                l₁⋅l₂⋅m₂⋅⎝g⋅m₁⋅sin(q₁) 
 l₂⋅m₂⋅⎜-g⋅l₂⋅m₂⋅sin(q₂) + l₁⋅l₂⋅m₂⋅u₁ ⋅sin(q₁ - q₂) + ───────────────────────
       ⎝                                           

In [35]:
T2_sol = sm.trigsimp(udr_sol[3])
T2_sol

                         ⎛                                                    
                       2 ⎜                              2                l₁⋅l₂
               l₁⋅l₂⋅m₂ ⋅⎜-g⋅l₂⋅m₂⋅sin(q₂) + l₁⋅l₂⋅m₂⋅u₁ ⋅sin(q₁ - q₂) + ─────
                         ⎝                                                    
g⋅m₂⋅cos(q₂) + ───────────────────────────────────────────────────────────────
                                                                              
                                                                              
                                                                      (l₁⋅m₁ +
                                                                              

    ⎛                                      2             ⎞             ⎞      
⋅m₂⋅⎝g⋅m₁⋅sin(q₁) + g⋅m₂⋅sin(q₁) + l₂⋅m₂⋅u₂ ⋅sin(q₁ - q₂)⎠⋅cos(q₁ - q₂)⎟      
───────────────────────────────────────────────────────────────────────⎟⋅sin(q
                          l₁⋅m₁ + l₁⋅m₂            

In [36]:
q0 = np.array([
    np.deg2rad(15.0),  # q1 [rad]
    np.deg2rad(25.0),  # q2 [rad]
])

u0 = np.array([
    np.deg2rad(123.0),  # u1 [rad/s]
    np.deg2rad(-41.0),  # u2 [rad/s]
])

p_vals = np.array([
    1.2,  # m1 [kg]
    5.6,  # m2 [kg]
    1.34,  # l1 [m]
    6.7,  # l2 [m]
    9.81,  # g [m/2^2]
])

In [37]:
eval_d = sm.lambdify((q, u, p), (Md, gd))
eval_a = sm.lambdify((q, u, p), (Ma, ga))

Md_vals, gd_vals = eval_d(q0, u0, p_vals)
Ma_vals, ga_vals = eval_a(q0, u0, p_vals)

In [38]:
-np.linalg.solve(Md_vals, np.squeeze(gd_vals))

array([  8.09538007,  -2.37332094, 109.88598116,  92.50997719])

In [39]:
-np.linalg.solve(Ma_vals, np.squeeze(ga_vals))

array([  8.09538007,  -2.37332094, 109.88598116,  92.50997719])

In [40]:
eval_forces = sm.lambdify((q, u, p), (T1_sol, T2_sol))
eval_forces(q0, u0, p_vals)

(109.88598116161896, 92.50997719098791)

In [41]:
def eval_rhs_newton(t, x, p):

    q = x[:2]
    u = x[2:]

    Md, gd = eval_d(q, u, p)
    udr = -np.linalg.solve(Md, np.squeeze(gd))

    qd = u
    ud = sol[:2]
    r = sol[2:]

    return np.hstack((qd, ud))